# Module 3.4: Incorporating the Color Sensor

Now, we will put together our knowledge about color sensors from 3.1 with our knowledge about loops from 3.3 to program the ride to respond to color-based signals.

Run the following code and connect to your SPIKE hub. 

In [3]:
import serial
import sys
import glob
import time
import serial.tools.list_ports
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output


ser = None

def serial_ports():
     result = []
     ports = serial.tools.list_ports.comports()
     for port, desc, hwid in sorted(ports):
          comm =  "{}: {}".format(port, desc)
          result.append(comm) 
     return result

def InitSerial(port, bps = 9600, to = 0):
    global ser
    try:
        ser = serial.Serial(port, bps, timeout = to)  # open serial port
        ser.flushInput()
        ser.flushOutput()
        return ser.name
    except Exception as e:
        return 'ERR: ' + str(e)

def CloseSerial():
    return('done')
    try:
        ser.flush()
        ser.close()
        return 'done'
    except Exception as e:
        return 'ERR: ' + str(e)    

def WriteSerial(string):
    try:
        reply = ser.write(string.encode())
        return str(reply)
    except Exception as e:
        return 'ERR: ' + str(e)    

def ReadSerial():
    try:
        reply = ''
        while ser.in_waiting:
            reply = reply + ser.readline().decode()
        return reply
    except Exception as e:
        return 'ERR: ' +  str(e)
    
def SendRead(string):
    if WriteSerial(string):
        time.sleep(0.1)
        return ReadSerial()
    
updatedPortList = []

def search_for_ports():
#     updatedPortList = []
    serialPortList = serial_ports()
    for x in serialPortList:
        updatedPortList.append(x[:x.index(':')])
        
def on_value_change(change):
    with output2:
        InitSerial(change['new'],115200)
        WriteSerial('\x03')
        run_some_code()
        
def run_some_code():
    code = '''
import hub,utime
    '''
    WriteSerial('\x05')
    WriteSerial(code)
    WriteSerial('\x04')
    WriteSerial('\x03')
    
    time.sleep(1) #wait for everything to get over there and read all the replies
    words = ReadSerial()
#     print(words)
    if (">>>" in words):
        print("You are now connected to SPIKE Prime!")
    else:
        print("Error, try reconnecting or connecting to a different serial port.")

updatedPortList = []
search_for_ports()
dropdown = widgets.Select(
            options=updatedPortList,
            description='Select Port:',
            disabled=False)
output2 = widgets.Output()
display(dropdown, output2)
dropdown.observe(on_value_change, names='value')

Select(description='Select Port:', options=('/dev/cu.Bluetooth-Incoming-Port', '/dev/cu.LEGOHub30E283048E8E', …

Output()

Next, load the functions that you may need in programming the ride. These include:
* move(port,rotations,speed) -- takes the port letter (ex. 'A'), number of motor rotations, and speed (between -100 and 100) as parameters and turns the motor accordingly.
* read() -- takes no parameters, returns the input from the serial port as a string. 
* setupColorSensor(port) -- takes the port letter (ex. 'A') as a parameter and connects to the sensor. It must be run before calling any function to get color sensor data.
* color(num) -- takes an integer as a parameter (2 for red, 3 for blue, 4 for green) and returns the intensity of the corresponding color (between 0 and 1024)
* getColors() -- returns red green and blue intensity values (between 0 and 1024) as a tuple
* getRGB() -- returns RGB values (between 0 and 1024) as a tuple
* printColor(rgb) -- takes an array of integers as a parameter and prints the hex value as well as a swatch of the corresponding color.
* showPixel(x,y) -- takes the x and y values of a pixel and lights the corresponding pixel on the hub.
* clearDisplay() -- turns off all pixels on the hub.

In [4]:
def move(port,rotations,speed):
    moveCode = """hub.port."""+str(port)+""".motor.run_for_degrees("""+str(360*rotations)+""", """+str(speed)+""")"""
    WriteSerial('\x05')
    WriteSerial(moveCode)
    WriteSerial('\x04')

def read():
    string1=''
    string2 = '  '
    while len(string2):
        time.sleep(0.1)
        string2 = ReadSerial()
        string1 = string1+string2
    return string1

def setupColorSensor(port):
    WriteSerial('import hub\r\n')
    WriteSerial('light_sensor = hub.port.'+str(port)+'.device\r\n')

def color(num):
    WriteSerial('print(light_sensor.get()['+str(num)+'])\r\n')
    return(int(read().split('\r\n')[-2]))

def getColors():
    return ((color(2),color(3),color(4)))

def getRGB():
    return(int(color(2)/1024*255),int(color(3)/1024*255),int(color(4)/1024*255))

def printColor(rgb):
    colors = ["#"+('%02x%02x%02x' %(rgb[0],rgb[1],rgb[2]))]
    display(Markdown('<br>'.join(
        f'<span style="font-family: monospace">{color} <span style="color: {color}">████████</span></span>'
        for color in colors
    )))

def showPixel(x,y):
    pixelCode = """hub.display.pixel("""+str(x)+""","""+str(y)+""",9)"""
    WriteSerial('\x05')
    WriteSerial(pixelCode)
    WriteSerial('\x04')

def clearDisplay():
    clearCode = """hub.display.clear()"""
    WriteSerial('\x05')
    WriteSerial(clearCode)
    WriteSerial('\x04')

Now, program your ride! Some specifications:
* you must move both of your motors in some sort of loop
* your motors must behave according to data from the color sensor (ex. green runs the ride, red stops the ride)
* your loops must have exit conditions (essentially, you must not put your code in a while(True) loop as this may cause the entire notebook to freeze)

### Congratulations, you've completed the third module! 
When you are finished with your ride, disassemble it. You're now ready to move on! 